In [1]:
import pandas as pd
from pymongo import MongoClient
import re
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
geolocator = Nominatim(user_agent="Krystian")

In [4]:
client = MongoClient('localhost', 27017)
db = client['test-database']
db.list_collection_names()
daneSurowe = db["Otodom"]
daneSurowe = list(daneSurowe.find())
daneSurowe = pd.DataFrame.from_dict(daneSurowe)

In [5]:
def zwracaCene(x):
    x = x.replace(' ','').replace(',','.')
    if len(re.findall(r'zł', x))>0:
        return float(x.replace('zł',''))
    else:
        return 4.54*float(x.replace('EUR',''))

In [6]:
adress = 'Kraków, Rynek Główny'
location = geolocator.geocode(adress)
lat1, lon1 = location[1][0],location[1][1]

In [7]:
def odlegloscOdCentrum(ulica):
    try:
        lat1,lon1 = (50.061516499999996, 19.935947926355013)
        location = geolocator.geocode(ulica.replace('ul.',''))
        lat2, lon2 = location[1][0],location[1][1]
        r = 6371
        phi1 = np.radians(lat1)
        phi2 = np.radians(lat2)
        delta_phi = np.radians(lat2-lat1)
        delta_lambda = np.radians(lon2-lon1)
        a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2)*np.sin(delta_lambda / 2)**2
        res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)))
    except TypeError as err :
        return float(0)
    else:
        return np.round(res, 2)

In [ ]:
daneSurowe.drop(daneSurowe[daneSurowe['Cena']=='Zapytaj o cenę'].index, axis=0, inplace=True)
daneSurowe.drop('_id', axis=1, inplace=True)
daneSurowe['Rynek'] = daneSurowe['Rynek'].apply(lambda x:  0 if x=='wtórny' else 1 )
daneSurowe['Cena'] = daneSurowe['Cena'].apply(zwracaCene)
daneSurowe['Powierzchnia'] = daneSurowe['Powierzchnia'].apply(lambda x:float(x.replace('m²','').replace(' ','').replace(',','.')))
daneSurowe['Liczba_pokoi'] = daneSurowe['Liczba_pokoi'].apply(lambda x: 11 if x =='więcej niż 10' else x)
daneSurowe['Winda'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'winda' in x else 0)
daneSurowe['Balkon'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'balkon' in x else 0)
daneSurowe['Klimatyzacja'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'klimatyzacja' in x else 0)
daneSurowe['Monitoring'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'monitoring' in x else 0)
daneSurowe['Internet'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'internet' in x else 0)
daneSurowe['Teren zamknięty'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'teren zamknięty' in x else 0)
daneSurowe['Piwnica'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'piwnica' in x else 0)
daneSurowe['Garaż/Miejsce parkingowe'] = daneSurowe['informacjeDodatkowe'].apply(lambda x: 1 if 'garaż/miejsce parkingowe' in x else 0)

In [ ]:
daneSurowe['Odległość'] = daneSurowe['Lokalizacja'].apply(odlegloscOdCentrum)